In [1]:
# %load train_kronnc.py
# random del 15 个nodes  email.txt
import torch
import pickle
import random
import numpy as np
import networkx as nx 
from networkx.convert import from_dict_of_dicts
from networkx.classes.graph import Graph
from kronEM import *
seed =1900
np.random.seed(seed)
random.seed(seed)

adj_address ="/data/chenmy/voter/seed1051email128-adjmat.pickle"
with open(adj_address,'rb') as f:
    objective_adj = pickle.load(f,encoding='latin1')
objective_adj = np.array(objective_adj)

In [2]:
Ground_truth_adj = torch.FloatTensor(objective_adj)
sz = Ground_truth_adj.shape[0]
k = int(np.log2(sz))
remove_proportion = 0.1
del_num = int(sz*remove_proportion)
mask_un_obs,mask_obs,miss_idx = missing_label(sz,remove_proportion)
G = Ground_truth_adj*mask_obs
nG = G.data.numpy()
H = G.clone()
# print(2,(abs(H.data.numpy()-objective_adj)*mask_obs.data.numpy()).sum())
init_z = Ground_truth_adj*mask_un_obs
missing_edges = int(init_z.sum())
print("miss nodes ",miss_idx)
# initial partial z with fixed missing edges num
z_ele_num = int(mask_un_obs.sum())
print("miss_edge",z_ele_num)
z_element_choice = torch.randperm(z_ele_num)[:missing_edges]
z_element = torch.nonzero(mask_un_obs)
init_z_edges = torch.index_select(z_element,0,z_element_choice)
H[init_z_edges[:,0],init_z_edges[:,1]] = 1
# print(abs((H - Ground_truth_adj)*mask_obs).sum())
# print(G,H,z_element_choice)
# initial kronecker
p0 = torch.FloatTensor([[0.9,0.7],[0.7, 0.3]])#torch.FloatTensor([[0.4408, 0.1770],[0.4951, 0.2585]])  # 
generator = kronecker_Generator(p0,k,2)
Pk = generator.generator_adjacency()
init_H = H.data.numpy()
label_non_obs = mask_un_obs.data.numpy()
init_Pk = Pk.detach().data.numpy()
perm = np.arange(sz)
print(abs((init_H- objective_adj)*mask_obs.data.numpy()).sum())
init_label_non_obs= label_non_obs.copy()

miss nodes  tensor([117,  82, 114,  11,  48,  37,  88,  90,  33,  75, 115,  14])
miss_edge 2928
0.0


In [2]:
# toy train data  generator
# 删除0.25比例的节点 origin: we randomly removed 25% of the nodes and the corresponding edges.
k = 7
sz = 2**k
remove_proportion = 0.1
del_num = int(sz*remove_proportion)

orip = torch.FloatTensor([[0.9,0.7],[0.5, 0.3]]) # torch.FloatTensor([[0.1981, 0.6427],[0.9684, 0.4522]])  
print("objective p", orip)
ground_adj = generator_adj(k,orip)

Ground_truth_adj_before = (ground_adj>torch.rand((ground_adj.shape))).float()
# sigma=Ground_truth_adj



baseline = loss_func(Ground_truth_adj_before,ground_adj,1)
print("base",baseline)

np_baseline = NLL(Ground_truth_adj_before.data.numpy(),ground_adj.data.numpy())
print("base2",np_baseline)

# shuffle 
node_num=sz
permu_m = torch.eye(node_num)
permutetion_arrange = torch.randperm(node_num)
permu_m_shuffle= torch.index_select(permu_m,0,permutetion_arrange) 

Ground_truth_adj = torch.mm(permu_m_shuffle,Ground_truth_adj_before).mm(permu_m_shuffle.t())

mask_un_obs,mask_obs,miss_idx = missing_label(sz,remove_proportion)
G = Ground_truth_adj*mask_obs
init_z = Ground_truth_adj*mask_un_obs
missing_edges = int(init_z.sum())

# initial partial z with fixed missing edges num
z_ele_num = int(mask_un_obs.sum())
z_element_choice = torch.randperm(z_ele_num)[:missing_edges]
z_element = torch.nonzero(mask_un_obs)
init_z_edges = torch.index_select(z_element,0,z_element_choice)
H = G.clone()
H[init_z_edges[:,0],init_z_edges[:,1]] = 1
# print(G,H,z_element_choice)
# initial kronecker

p0 = torch.rand([2,2])  # torch.FloatTensor([[0.4408, 0.1770],[0.4951, 0.2585]]) 
generator = kronecker_Generator(p0,k,2)
Pk = generator.generator_adjacency()
init_H = H.data.numpy()
label_non_obs = mask_un_obs.data.numpy()
init_Pk = Pk.detach().data.numpy()
init_label_non_obs= label_non_obs.copy()

objective p tensor([[0.9000, 0.7000],
        [0.5000, 0.3000]])
base tensor(1761.8107)
base2 1761.811


In [3]:
def gumbel_sample(shape, eps=1e-20):
    u = torch.rand(shape)
    gumbel = - np.log(- np.log(u + eps) + eps)
    if use_cuda:
        gumbel = gumbel.cuda()
    return gumbel

def gumbel_softmax_sample(logits, temperature):
    """ Draw a sample from the Gumbel-Softmax distribution"""
    # gumbel_sample 返回一个sample采样
    y = logits + gumbel_sample(logits.size())
    return torch.nn.functional.softmax(y/temperature, dim=1)

def gumbel_softmax(logits, temperature, hard=False):
    """Sample from the Gumbel-Softmax distribution and optionally discretize.
    Args:
    logits: [batch_size, n_class] unnormalized log-probs
    temperature: non-negative scalar
    hard: if True, take argmax, but differentiate w.r.t. soft sample y
    Returns:
    [batch_size, n_class] sample from the Gumbel-Softmax distribution.
    If hard=True, then the returned sample will be one-hot, otherwise it will
    be a probabilitiy distribution that sums to 1 across classes
    """
    
    y = gumbel_softmax_sample(logits, temperature)
    
    if hard:
        k = logits.size()[-1]
        y_hard = torch.max(y.data, 1)[1]
        y = y_hard
    return y


In [4]:
# %load kronEM.py
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,auc
import numpy as np

def kronecker(A,B):
    return torch.einsum("ab,cd->acbd", A, B).view(A.size(0)*B.size(0),  A.size(1)*B.size(1))
class kronecker_Generator(nn.Module):
    def __init__(self,p0,korder = 3,node_num = 2):
        super(kronecker_Generator,self).__init__()
        self.p = Parameter(p0,requires_grad = True)
        # self.p = Parameter(torch.rand(node_num,node_num,requires_grad=True))
        self.korder = korder
        # print(self.p)
    def generator_adjacency(self):
        k = self.korder
        p0 = self.p
        adj = self.p
        for i in range(k-1):
            adj = kronecker(adj,p0)
        return adj
def loss_func(sigma,Pk):
    loss = -torch.sum((1-sigma)*torch.log(1-Pk)+sigma*torch.log(Pk))
    return loss
def metropolis_update_ratio(sigma_before,sigma_later,Pk):
    '''
    if memory is sufficinet, this one is much cleaner to execute
    '''
    Nll_before=(1-sigma_before)*np.log(1-Pk) +sigma_before*np.log(Pk)
    Nll_later=(1-sigma_later)*np.log(1-Pk) +sigma_later*np.log(Pk)
    ratio=np.exp(np.sum(Nll_later-Nll_before))
    return ratio
def SwapElement(sigma_before,i,j):
    i_topology=sigma_before[i,:]
    j_topology=sigma_before[j,:]
    sigma_later=np.copy(sigma_before)
    sigma_later[i,:]=j_topology
    sigma_later[j,:]=i_topology
    sigma_later[:,i]=sigma_before[:,j]
    sigma_later[:,j]=sigma_before[:,i]
    sigma_later[i,j]=sigma_before[i,j]
    sigma_later[j,i]=sigma_before[j,i]
    sigma_later[i,i]=sigma_before[j,j]
    sigma_later[j,j]=sigma_before[i,i]
    return sigma_later

def SamplePermutation(Pk,sigma,u,n1_swap,n2_swap,label_non_obs,obj_adj):
    sigma_later=SwapElement(sigma,n1_swap, n2_swap)
    ratio=metropolis_update_ratio(sigma,sigma_later,Pk)
    if u<ratio:
        sigma=sigma_later
        label_non_obs=SwapElement(label_non_obs,n1_swap, n2_swap)
        obj_adj = SwapElement(obj_adj,n1_swap,n2_swap)
#         print(ifswap)
    return sigma,label_non_obs,obj_adj
def SampleZ(H,Pk,label_non_obs,u):
    mat_size=len(Pk)
    edge_in_non_obs=(H>0)*label_non_obs  
    edge_position=np.where(edge_in_non_obs)
    edge_removed=np.random.randint(len(edge_position[0]))
    px=Pk[edge_position[0][edge_removed],edge_position[1][edge_removed]]
    non_edge_in_non_obs=(H<1)*label_non_obs
    ##return 
    py_array=non_edge_in_non_obs*Pk
    py_array=np.ravel(py_array)
    py=np.random.choice(range(len(py_array)), size=1, p=py_array/np.sum(py_array))
    ratio=(1-py_array[py])/(1-px)
    if ratio<u:
        H[py//mat_size,py%mat_size]=1
        H[edge_position[0][edge_removed],edge_position[1][edge_removed]]=0
        # print('accept')
    return H

def missing_label(sz,missing_percent):
     # random sample
    missing_num = int(sz*missing_percent)
    idx = torch.randperm(sz)[:missing_num]
    mask_un_obs = torch.zeros(sz,sz)
    for i in idx:
        mask_un_obs[i] = 1
        mask_un_obs[:,i:i+1] = 1
    mask_obs = 1- mask_un_obs
    return mask_un_obs,mask_obs,idx
# only sigma and kronecker pk are itered
def E_step(sigma,N,warmup,Pk,label_non_obs,obj_adj):
    u1=np.random.rand(N+warmup)
    u2=np.random.rand(N+warmup)
    sigma_hist=[]
    Z_label=[]
    Node_list=np.arange(len(Pk))
    element_to_swap=np.random.choice(a=Node_list,size=(2,3*(N+warmup)))
    mask=element_to_swap[1,:]!=element_to_swap[0,:]# it is pointless to swap the same element
    n1_swap=element_to_swap[0,:][mask]
    n2_swap=element_to_swap[1,:][mask]
    
    for i in range(warmup):
        sigma,label_non_obs,obj_adj=SamplePermutation(Pk,sigma,u2[i],n1_swap[i],n2_swap[i],label_non_obs,obj_adj)  
    for j in range(warmup):
        sigma=SampleZ(sigma,Pk,label_non_obs,u1[i+j:i+j+1]) 
    for k in range(N):
        sigma=SampleZ(sigma,Pk,label_non_obs,u1[i+j+1+k:i+j+1+k+1]) 
        sigma_hist.append(sigma)
    Z_label.append(label_non_obs)
    return sigma_hist,Z_label,obj_adj #（H+G）

def M_step(epoch,sigma_train,p0,k,N):
    losses = []
    generator = kronecker_Generator(p0,k,2)
    learning_rate = 1e-5#0.0000001
    opt_net = optim.SGD(generator.parameters(),lr = learning_rate)
    decayRate = 0.95

    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=opt_net, gamma=decayRate) 
    for i in range(epoch):   
        opt_net.zero_grad()
        Pk = generator.generator_adjacency()
        loss = loss_func(sigma_train,Pk,N)
        loss2 = loss2_func(sigma_train.detach().numpy(),Pk.detach().numpy(),N)
        loss.backward()
        losses.append(loss.item())
        # print(str(i),loss.item(),loss2.item(),(loss2-loss).item())
        opt_net.step()
        scheduler.step()
        for group in opt_net.param_groups:
            for param in group["params"]: 
              # print("before param",param)
              param.data.clamp_(0.0001,0.9999)
              # print("after param",param)
    
        for p in generator.parameters():
            p0 = p.data  
        generator = kronecker_Generator(p0,k,2)
        Pk = generator.generator_adjacency()
        Pk = Pk.detach().numpy() 
        # evaluation
        #   nll infer  nll true
    return np.mean(losses),Pk,p0

def kronEM(iterstep,N,warmup,H,Pk,label_non_obs,epoch,p0,obj_adj,k):
    emlosses = []
    for i in range(iterstep):
        print("start iterstep",i)
        sigma_hist,Z_label,obj_adj = E_step(H,N,warmup,Pk,label_non_obs,obj_adj)
        print("E-step")
        H = sigma_hist[-1]
        label_non_obs = Z_label[-1]
        sigma_hist_train = np.array(sigma_hist)
        sigma_train = torch.DoubleTensor(sigma_hist_train)
        emloss,Pk,p0 = M_step(epoch,sigma_train,p0,k,N)
        emlosses.append(emloss)
        print("*************\n EM loss is %f"%emloss,"p0 is ",p0)
        
        # evaluation
        inferp_nll = NLL(H,Pk)
        print("inferp_nll is %f "%(inferp_nll))
        obs_mask = (1-label_non_obs).astype(bool)
        obs_auc = calauc(obj_adj,Pk,obs_mask)
        non_obs_auc = calauc(obj_adj,Pk,label_non_obs.astype(bool))
        #         print("perm is ", perm)
        
        label_obs = (1-label_non_obs)
        obs_diff_edge = abs((H - obj_adj)*label_obs).sum()
        unobs_diff_edge =  abs((H - obj_adj)*label_non_obs).sum()
        all_diff_edge = abs((H - obj_adj)).sum()
        # D_abs = torch.abs(orip-p0).mean()
        print("infer p is ",p0,  "obseved auc  is %f and non_obs auc is %f and obs_diff_edge is %f and un_obs_diff_edge %f and all_diff_edge is %f"%(obs_auc,non_obs_auc,obs_diff_edge,unobs_diff_edge,all_diff_edge))
    
    return emlosses,H,Pk

def NLL(sigma_true,pk):
    Nll_before=(1-sigma_true)*np.log(1-pk) +sigma_true*np.log(pk)
    return -np.sum(Nll_before)

def loss2_func(sigma_train,Pk,N):
    loss2 = 0
    for i in range(N):
        loss2+= NLL(sigma_train[i],Pk)
    # print(loss2)
    return loss2/N

def loss_func(sigma,Pk,N):
    loss = -torch.sum((1-sigma)*torch.log(1-Pk)+sigma*torch.log(Pk))
    return loss/N

def generator_adj(korder,p):
    k = korder
    p0 = p
    adj = p
    for i in range(k-1):
        adj = kronecker(adj,p0)
    return adj
def calauc(H,Pk,mask):
    fpr, tpr, thresholds = roc_curve(H[mask],Pk[mask])
    Auc = auc(fpr, tpr)
    return Auc

In [ ]:
warmup=300
N = 200
iterstep = 50
epoch = 20
emlosses2,H,pk= kronEM(iterstep,N,warmup,init_H,init_Pk,label_non_obs,epoch,p0,objective_adj,k)

In [ ]:
warmup=3000
N = 1500
iterstep = 100
epoch = 50
emlosses2,H,pk= kronEM(iterstep,N,warmup,init_H,init_Pk,label_non_obs,epoch,p0,objective_adj,k)

start iterstep 0
E-step
*************
 EM loss is 1880.732760 p0 is  tensor([[0.6497, 0.7507],
        [0.6418, 0.3430]])
inferp_nll is 1880.726196 
infer p is  tensor([[0.6497, 0.7507],
        [0.6418, 0.3430]]) obseved auc  is 0.507791 and non_obs auc is 0.555585 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 198.000000 and all_diff_edge is 1416.000000
start iterstep 1
E-step
*************
 EM loss is 1888.114944 p0 is  tensor([[0.6522, 0.7510],
        [0.6402, 0.3417]])
inferp_nll is 1888.093872 
infer p is  tensor([[0.6522, 0.7510],
        [0.6402, 0.3417]]) obseved auc  is 0.441396 and non_obs auc is 0.531746 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 198.000000 and all_diff_edge is 1416.000000
start iterstep 2
E-step
*************
 EM loss is 1870.849295 p0 is  tensor([[0.6553, 0.7515],
        [0.6382, 0.3398]])
inferp_nll is 1870.815674 
infer p is  tensor([[0.6553, 0.7515],
        [0.6382, 0.3398]]) obseved auc  is 0.500678 and non_obs auc is 0.530569 and

In [5]:
warmup=30
N = 15
iterstep = 100
epoch = 50
emlosses2,H,pk= kronEM(iterstep,N,warmup,init_H,init_Pk,label_non_obs,epoch,p0,objective_adj,k)

start iterstep 0
E-step
*************
 EM loss is 2434.850030 p0 is  tensor([[0.4206, 0.6819],
        [0.5978, 0.3032]])
inferp_nll is 2432.945557 
infer p is  tensor([[0.4206, 0.6819],
        [0.5978, 0.3032]]) obseved auc  is 0.484402 and non_obs auc is 0.523732 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 198.000000 and all_diff_edge is 1416.000000
start iterstep 1
E-step
*************
 EM loss is 2336.690232 p0 is  tensor([[0.4334, 0.6890],
        [0.6070, 0.3245]])
inferp_nll is 2335.270996 
infer p is  tensor([[0.4334, 0.6890],
        [0.6070, 0.3245]]) obseved auc  is 0.490670 and non_obs auc is 0.534361 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 198.000000 and all_diff_edge is 1416.000000
start iterstep 2
E-step
*************
 EM loss is 2251.935303 p0 is  tensor([[0.4454, 0.6957],
        [0.6153, 0.3417]])
inferp_nll is 2250.886230 
infer p is  tensor([[0.4454, 0.6957],
        [0.6153, 0.3417]]) obseved auc  is 0.486320 and non_obs auc is 0.555070 and

E-step
*************
 EM loss is 1964.946328 p0 is  tensor([[0.5551, 0.7421],
        [0.6676, 0.4144]])
inferp_nll is 1964.929321 
infer p is  tensor([[0.5551, 0.7421],
        [0.6676, 0.4144]]) obseved auc  is 0.497654 and non_obs auc is 0.551505 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 202.000000 and all_diff_edge is 1420.000000
start iterstep 24
E-step
*************
 EM loss is 1955.008454 p0 is  tensor([[0.5577, 0.7426],
        [0.6671, 0.4130]])
inferp_nll is 1954.989990 
infer p is  tensor([[0.5577, 0.7426],
        [0.6671, 0.4130]]) obseved auc  is 0.501302 and non_obs auc is 0.564280 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 202.000000 and all_diff_edge is 1420.000000
start iterstep 25
E-step
*************
 EM loss is 1956.048946 p0 is  tensor([[0.5603, 0.7431],
        [0.6664, 0.4118]])
inferp_nll is 1956.031860 
infer p is  tensor([[0.5603, 0.7431],
        [0.6664, 0.4118]]) obseved auc  is 0.495791 and non_obs auc is 0.568640 and obs_diff_edge 

E-step
*************
 EM loss is 1929.570234 p0 is  tensor([[0.6078, 0.7466],
        [0.6558, 0.3756]])
inferp_nll is 1929.553101 
infer p is  tensor([[0.6078, 0.7466],
        [0.6558, 0.3756]]) obseved auc  is 0.521326 and non_obs auc is 0.609262 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 200.000000 and all_diff_edge is 1418.000000
start iterstep 47
E-step
*************
 EM loss is 1925.529722 p0 is  tensor([[0.6098, 0.7465],
        [0.6553, 0.3742]])
inferp_nll is 1925.517456 
infer p is  tensor([[0.6098, 0.7465],
        [0.6553, 0.3742]]) obseved auc  is 0.523402 and non_obs auc is 0.613968 and obs_diff_edge is 1218.000000 and un_obs_diff_edge 200.000000 and all_diff_edge is 1418.000000
start iterstep 48
E-step
*************
 EM loss is 1927.613210 p0 is  tensor([[0.6117, 0.7465],
        [0.6548, 0.3729]])
inferp_nll is 1927.602539 
infer p is  tensor([[0.6117, 0.7465],
        [0.6548, 0.3729]]) obseved auc  is 0.510075 and non_obs auc is 0.578106 and obs_diff_edge 

KeyboardInterrupt: 

In [9]:
remove_para =  label_non_obs.sum()

In [ ]:
def kronecker(A,B):
    return torch.einsum("ab,cd->acbd", A, B).view(A.size(0)*B.size(0),  A.size(1)*B.size(1))
class kronecker_Generator(nn.Module):
    def __init__(self,p0,korder = 3,node_num = 2):
        super(kronecker_Generator,self).__init__()
        self.p = Parameter(p0,requires_grad = True)
        # self.p = Parameter(torch.rand(node_num,node_num,requires_grad=True))
        self.korder = korder
        # print(self.p)
    def generator_adjacency(self):
        k = self.korder
        p0 = self.p
        adj = self.p
        for i in range(k-1):
            adj = kronecker(adj,p0)
        return adj

In [ ]:
class Gumbel_union_kron(nn.Module):
    def __init__(self,p0,korder,perm,label_non_obs,temp=10,temp_drop_frac = 0.9999):
        super(Gumbel_union_kron).__init__()
        self.p = Parameter(p0,requires_grad = True)
        self.korder = korder
        self.perm = perm
        self.label_non_obs = label_non_obs
    def generator_adjacency(self):
        k = self.korder
        p0 = self.p
        adj = self.p
        for i in range(k-1):
            adj = kronecker(adj,p0)
        return adj
    
    def sample_all(self,hard=False):
        
        self.logp = self.gen_matrix
        if use_cuda:
            self.logp = self.gen_matrix.cuda()
        
        out = gumbel_softmax(self.logp, self.temperature, hard)
        if hard:
            hh = torch.zeros((self.del_num*(2*self.sz - self.del_num-1),2))
            for i in range(out.size()[0]):
                hh[i, out[i]] = 1
            out = hh                    
        out = out[:, 0]
        if use_cuda:
            out = out.cuda()
            
        matrix = torch.zeros(self.sz,self.sz).cuda()
        left_mask = torch.ones(self.sz,self.sz)
        left_mask[:-self.del_num,:-self.del_num] = 0
        left_mask = left_mask - torch.diag(torch.diag(left_mask))
        un_index = left_mask.nonzero()
        matrix[(un_index[:,0],un_index[:,1])] = out
        out_matrix = matrix
        # out_matrix = out[:, 0].view(self.gen_matrix.size()[0], self.gen_matrix.size()[0])
        return out_matrix
    def init(self, mean, var):
        init.normal_(self.gen_matrix, mean=mean, std=var)
        
    
    
    
    

In [ ]:
'''在网络补全任务中生成未知部分结构'''
class Gumbel_Generator_nc_asy(nn.Module):
    def __init__(self, sz=10,del_num = 1,temp=10, temp_drop_frac=0.9999):
        super(Gumbel_Generator_nc_asy, self).__init__()
        self.sz = sz
        self.del_num = del_num
        self.gen_matrix = Parameter(torch.rand(del_num*(2*sz-del_num-1), 2)) #cmy get only unknown part parameter
        self.temperature = temp
        self.temp_drop_frac = temp_drop_frac

    def drop_temp(self):
        # 降温过程
        self.temperature = self.temperature * self.temp_drop_frac

    def sample_all(self, hard=False):
        self.logp = self.gen_matrix
        if use_cuda:
            self.logp = self.gen_matrix.cuda()
        
        out = gumbel_softmax(self.logp, self.temperature, hard)
        if hard:
            hh = torch.zeros((self.del_num*(2*self.sz - self.del_num-1),2))
            for i in range(out.size()[0]):
                hh[i, out[i]] = 1
            out = hh                    
        out = out[:, 0]
        if use_cuda:
            out = out.cuda()
            
        matrix = torch.zeros(self.sz,self.sz).cuda()
        left_mask = torch.ones(self.sz,self.sz)
        left_mask[:-self.del_num,:-self.del_num] = 0
        left_mask = left_mask - torch.diag(torch.diag(left_mask))
        un_index = left_mask.nonzero()
        matrix[(un_index[:,0],un_index[:,1])] = out
        out_matrix = matrix
        # out_matrix = out[:, 0].view(self.gen_matrix.size()[0], self.gen_matrix.size()[0])
        return out_matrix
    def init(self, mean, var):
        init.normal_(self.gen_matrix, mean=mean, std=var)

In [20]:
## union code 
kronfit 已知部分 + gumbel   + 两个loss （ll+states loss）

0.0

In [15]:
A = np.random.rand(3,3)

In [29]:
A = np.random.randint(2,size = ([3,3]))

In [23]:
perm_mat = np.array([0,2,1])

In [30]:
A

array([[1, 1, 1],
       [0, 1, 0],
       [1, 0, 1]])

In [13]:
perm

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  85,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  65,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        37,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  22,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127])

In [16]:
permutation_mat  = np.eye(H.shape[0])[perm]
shuffle_H = np.dot(np.dot(permutation_mat,objective_adj),permutation_mat.T)

In [10]:
sigma_later1=SwapElement(init_H,37,65)

In [8]:
sigma_later=SwapElement(sigma_later,22,85)

In [12]:
(abs(sigma_later1-H)*mask_obs.data.numpy()).sum()

68.0

In [18]:
(abs(shuffle_H-H)*mask_obs.data.numpy()).sum()

0.0